# <center> COVID'19 Tweets Geographical Distribution 🌎</center>

<center><h5>Recovered more than 70% of Geo Cordinates from user tweeted address</h5></center>

<center><img src='https://d25yuvogekh0nj.cloudfront.net/2019/08/Twitter-Banner-Size-Guide-blog-banner-1250x500.png'></center>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Dataset Description

These tweets are collected using Twitter API and a Python script. A query for this high-frequency hashtag (#covid19) is run on a daily basis for a certain time period, to collect a larger number of tweets samples.

**Content**
The tweets have #covid19 hashtag. Collection started on 25/7/2020, with an initial 17k batch and will continue on a daily basis.


The collection script can be found here: https://github.com/gabrielpreda/covid-19-tweets

## Importing Necesssary Libraries

In [ ]:
#importing necessery libraries for future analysis of the dataset
!pip install calmap

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import folium
from folium.plugins import HeatMapWithTime, TimestampedGeoJson
import matplotlib.style as style 
style.use('fivethirtyeight')
import numpy as np; np.random.seed(sum(map(ord, 'calmap')))
import pandas as pd
import calmap

# Dataset Exploration

In [ ]:
# Pandas to read Covid Tweets dataset
tweets = pd.read_csv('../input/covid19-tweets/covid19_tweets.csv')

In [ ]:
# Examining the dataset from begining
tweets.head()

In [ ]:
# Checking the shape of dataset
tweets.shape

In [ ]:
tweets.info()

# Data Insights
We have around 166,656 Tweets in the dataset and each tweet will possibly contain 13 parameters. Lets try to explore how many of them are necessary for visualizing the geographical distribution. To visualize geographically we need to have either country codes or Location co-ordinates. But te dataset has a parameter '*user_location*' in which the locations are very vague to plot them.


Main idea is to extract geographical cordinates of user. Here I'm using two other datasets to improve the location feature in dataset.
* world-cities-datasets (https://www.kaggle.com/viswanathanc/world-cities-datasets)

world-cities-datasets is used to identify Alpha 2,Alpha 3,Country Name, City Name from user location field in Covid Tweet Dataset.

In [ ]:
# World City Dataset

cities = pd.read_csv('../input/world-cities-datasets/worldcities.csv')

In [ ]:
# Exploring city dataset
cities.head()

In [ ]:
## Duplicate Location in Tweets Dataset

tweets["location"] = tweets["user_location"]
tweets["country"] = np.NaN

## Missing Data Handling in tweets dataset

Here i dont want to remove the NaN values now. So i'm trying to replace it with a empty string.

In [ ]:
user_location = tweets['location'].fillna(value='').str.split(',')

# Feature Engineering

In [ ]:
lat = cities['lat'].fillna(value = '').values.tolist()
lng = cities['lng'].fillna(value = '').values.tolist()
country = cities['country'].fillna(value = '').values.tolist()

# Getting all alpha 3 codes into  a list
world_city_iso3 = []
for c in cities['iso3'].str.lower().str.strip().values.tolist():
    if c not in world_city_iso3:
        world_city_iso3.append(c)
        
# Getting all alpha 2 codes into  a list    
world_city_iso2 = []
for c in cities['iso2'].str.lower().str.strip().values.tolist():
    if c not in world_city_iso2:
        world_city_iso2.append(c)
        
# Getting all countries into  a list        
world_city_country = []
for c in cities['country'].str.lower().str.strip().values.tolist():
    if c not in world_city_country:
        world_city_country.append(c)

# Getting all amdin names into  a list
world_states = []
for c in cities['admin_name'].str.lower().str.strip().tolist():
    world_states.append(c)


# Getting all cities into  a list
world_city = cities['city'].fillna(value = '').str.lower().str.strip().values.tolist()



# Data Mapping among Datasets

1. Each User Location may possibly contain combination of city name, country name with country codes. (Ex: Pewee Valley, KY)
2. We need to split the user location from user location
3. Now each location will be possibily containing list of (country, city, code)
4. If we find city name or admin name match in world city database we will assign the location from world city database to tweets dataset for a respective row.

In [ ]:

for each_loc in range(len(user_location)):
    ind = each_loc
    each_loc = user_location[each_loc]
    for each in each_loc:
        each = each.lower().strip()
        if each in world_city:
            order = world_city.index(each)
            tweets['country'][ind] = country[order]
            continue
        if each in world_states:
            order= world_states.index(each)
            tweets['country'][ind] = country[order]
            continue
        if each in world_city_country:
            order = world_city_country.index(each)
            tweets['country'][ind] = world_city_country[order]
            continue
        if each in world_city_iso2:
            order = world_city_iso2.index(each)
            tweets['country'][ind] = world_city_country[order]
            continue
        if each in world_city_iso3:
            order = world_city_iso3.index(each)
            tweets['country'][ind] = world_city_country[order]
            continue


# Data Analysis

In [ ]:
print('Total Number of valid Tweets Available: ',tweets['country'].isnull().sum())

## Top 20 Countries with Most Tweets 

In [ ]:
tweet_per_country = tweets['country'].str.lower().dropna()
tw = tweet_per_country.value_counts().rename_axis('Country').reset_index(name='Tweet Count')
print(tw)
plt.rcParams['figure.figsize'] = (15,10)
plt.title('Top 20 Countries with Most Tweets',fontsize=15)
sns.set_palette("husl")
ax = sns.barplot(y=tw['Country'].head(20),x=tw['Tweet Count'].head(20))